In [1]:
import os 

In [2]:
curr_dir = os.getcwd()
print("Current directory:",curr_dir)
os.chdir(r'../')
print("Changing to root directory:",curr_dir)

Current directory: c:\Users\rsurs\OneDrive\Documents\chest-cancer-net\research
Changing to root directory: c:\Users\rsurs\OneDrive\Documents\chest-cancer-net\research


In [3]:
from dataclasses import dataclass 
from pathlib import Path

# frozen makes the dataclass immutable
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path  
    unzip_dir: Path


'''
local_data_file visualized
artifacts/
└── data_ingestion/     ← root_dir
    ├── data.zip        ← local_data_file
    └── raw_data/       ← unzip_dir
'''

'\nlocal_data_file visualized\nartifacts/\n└── data_ingestion/     ← root_dir\n    ├── data.zip        ← local_data_file\n    └── raw_data/       ← unzip_dir\n'

In [4]:
from src.cancer_clf.constants.constant import *
from src.cancer_clf.utils.common import read_yaml,create_directories 

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        # This creates the artifacts folder
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 
        # this will pull up data_ingestion section from config.yaml file 
        # data_ingestion:
        #     root_dir: artifacts/data_ingestion
        #     source_URL: https://example.com/Chest-CT-Scan-data.zip
        #     local_data_file: artifacts/data_ingestion/data.zip
        #     unzip_dir: artifacts/data_ingestion/raw_data

        

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [6]:
import os 
import zipfile 
import gdown 
from src.cancer_clf.logger.logger import logger 
from src.cancer_clf.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str: 
        '''
        Fetches the data from url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extract the zip file into the data directory
        '''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-23 21:38:02,885: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-23 21:38:02,885: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-23 21:38:02,889: INFO: common: created directory at: artifacts]
[2026-01-23 21:38:02,890: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-23 21:38:02,892: INFO: 739917935: Downloading data from https://drive.google.com/file/d/166i44CxICqxH3bmvXpKUNy8ciLqVUeZr/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=166i44CxICqxH3bmvXpKUNy8ciLqVUeZr
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=166i44CxICqxH3bmvXpKUNy8ciLqVUeZr&confirm=t&uuid=084ea0cf-18da-47fb-8513-71fc29b2b1bd
To: c:\Users\rsurs\OneDrive\Documents\chest-cancer-net\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:00<00:00, 53.2MB/s]

[2026-01-23 21:38:06,958: INFO: 739917935: Downloaded data from https://drive.google.com/file/d/166i44CxICqxH3bmvXpKUNy8ciLqVUeZr/view?usp=drive_link into artifacts/data_ingestion/data.zip]
